# Introduction to the Memory Data

##### This is a script to load and work with data that was collected to analyze memorability of sounds.  Please see the below paper for details.

#### please cite:

Ramsay, David, Ishwarya Ananthabhotla, and Joseph Paradiso. "The Intrinsic Memorability of Everyday Sounds." Audio Engineering Society Conference: 2019 AES International Conference on Immersive and Interactive Audio. Audio Engineering Society, 2019.

### Getting Started

This will just demonstrate how to load the included data for use in your analysis.


We have 2 sets of data; (1) memory games associated with Amazon Turk user ids, and (2) surveys to collect background information from a unique token we track using cookies.

Turk IDs from Amazon are called *WorkerIDs* and are an uppercase string that starts with A and is 14 characters long.
Our user ID is called *uid* and is a 32 character string.

It turns out some workers log out and log in as 'different people', but they don't refresh their browser so they retain our UID across multiple WorkerIDs.  There are others that might have refreshed the browser and gotten a new UID despite the same WorkerID.  We also have a lot of people that might've started the task but never finished it, filling out a survey and getting a UID without ever getting far enough to trigger data collection and a WorkerID.


In [2]:
import pandas as pd

## Processed Memory/Confusability Scores for Simple Analysis

Most people will just want *memory scores* (how likely to be selected when it was played 60 samples ago) and *confusability* scores (how likely to be selected when it was not presented) associated with each sample; we've done that.  To make confusability a bit more reliable, we focus on a version calculated when a sound falls in the last 10 samples of the test (after hearing 60 other random sounds) for its first presentation, as this is closer to the conditions under which memory is being assessed for the target sounds.  Certainly you are more likely to confuse a sound after hearing 60 other sounds than hearing none.

In the data you can choose whether to view data from all games, or data only from games that pass a vigilance criteria of a false positive rate for new sounds < 0.4 and a vigilance sound second presentation true positive rate > 0.6 (filtered games).  In our paper, we consider memorability normalized by confusability (in other words, how much does the presentation of the sound earlier *increase the likelihood* that you will click it?  It's possible that some sounds are ambiguous and people naturally *think* they've heard them before; these will have a high memorability score even though they are definitely *not* memorable sounds.  To address this, we subtract the confusability score from the memorability score before trying to assess any trends in the data.

```
Confusability = how likely you are to click on a sound when you haven't heard before
Confusability_last10 = how likely you are to click on a sound when you haven't heard it before, in the last 10 presentations (having heard 60 other sounds)
Memorability = how likely you are to remember a sound that is separated by 60 other random sounds
Memorability_norm = Memorability - Confusability
Memorability_norm_last10 = Memorability - Confusability_last10
```

It may also be possible to draw some conclusions given the vigilance score, which has hundreds of presentations, but are only separated by 2-3 other sounds.

#### loading memory scores

In [24]:
memory_scores = pd.read_pickle('memory_scores_by_sound.pkl')
memory_scores.head()

,filename,t_correct,t_trials,v_correct,v_trials,confusability,confusability_last10,t_percent,v_percent,norm_mem,norm_mem_last10
0,(electric)_razor.wav,29,50,396,450,0.214286,0.250000,0.580000,0.880000,0.365714,0.330000
1,(lawn)_sprinkler.wav,23,48,387,429,0.160612,0.240000,0.479167,0.902098,0.318555,0.239167
2,(sliding)_chair.wav,26,46,352,453,0.142586,0.165354,0.565217,0.777042,0.422632,0.399863
3,(tornado)_siren.wav,19,46,418,447,0.089509,0.142857,0.413043,0.935123,0.323534,0.270186
4,(winding)_clock.wav,24,47,387,415,0.160481,0.201681,0.510638,0.932530,0.350157,0.308958


## Raw Memory Data, Survey Data and User IDs

for those looking to do more advanced analysis, we can also look at individual memory game data in raw form.  We can compare this against survey data from each particpant (or examine survey data alone).

As noted above, cross-validation of who is who is quite messy (Turkers changing their Turk ID while retaining our cookie/ID in browser, rejecting the cookie with the same Turk ID, not making it far enough after doing the survey and creating our cookie/ID).  We've done our best to match these IDs, and you'll find a single USER_ID column which is our best matching of Amazon IDs (which frequently have many of our user ids), falling back to our IDs if there is no Amazon match.  There might be a little bit of noise in the user matching, but we've done the hard work of cross-referencing submitted codes to Turk and browser tokens to try and uniquely identify individuals.  This ID is constant across survey results and game data.

We also collected a small amount of test data from MIT community members through a different portal.  The email field will show 'mturker' if they were directed to the game through Amazon Turk, and either 'NA' if they filled out the survey anonymously or 'XXX.mit.edu' if they filled it out with their email (incentivized by a giftcard).  No submitted emails were the same.

For the user surveys, answers capture the amount of waking time allocated for each location/activity:

```
0 = never
1 = once a month
2 = once a week
3 = a little each day
4 = many hours each day
```

and their situation was marked as 

```
0 = urban
1 = suburban
2 = rural
```


#### loading raw games

In [49]:
raw_games = pd.read_pickle('raw_game_data.pkl')
raw_games.head()

,tCorrect,vPercent,falsePositives,tLocation,fileList,guesses,received,USER_ID
0,[True],0.90,0.045455,"[0, 60]",[https://keyword.media.mit.edu/shared/ambiguou...,"[0, 0, 0, 0, 0, 3072, 0, 1164, 871, 0, 0, 0, 2...",2018-02-15T00:15:02.107Z,OtvPENAdy5wrsSZBGyKrblWraTHKds03
1,[False],1.00,0.159091,"[1, 61]",[https://keyword.media.mit.edu/shared/final_mo...,"[0, 0, 0, 866, 0, 0, 1160, 0, 0, 0, 0, 0, 0, 0...",2018-02-23T21:13:27.340Z,A2J2FGVGQCXAPB
2,[True],0.70,0.136364,"[2, 62]",[https://keyword.media.mit.edu/shared/natural_...,"[0, 0, 0, 0, 1332, 0, 0, 0, 1822, 0, 3697, 229...",2018-02-23T21:13:46.800Z,A24NDC7JH9LEM1
3,[True],1.00,0.068182,"[4, 64]",[https://keyword.media.mit.edu/shared/final_mo...,"[0, 0, 0, 0, 0, 1791, 3162, 0, 0, 0, 0, 0, 0, ...",2018-02-23T21:13:53.842Z,AV8YCU0R4D94I
4,[False],0.95,0.068182,"[1, 61]",[https://keyword.media.mit.edu/shared/natural_...,"[0, 0, 0, 0, 1064, 0, 0, 1942, 0, 0, 1057, 0, ...",2018-02-23T21:14:57.111Z,A1DVCUCOTAC0JQ


In [52]:
## we suggest filtering these, as we did in our publication, for people who paid attention

FALSE_NEG_MAX = 0.4
VIGIL_POS_MIN = 0.6

filtered_games = raw_games.loc[(raw_games['falsePositives'] < FALSE_NEG_MAX) & (raw_games['vPercent'] > VIGIL_POS_MIN)]

print('This operation filters the data from ' + str(len(raw_games)) + ' to ' + str(len(filtered_games)) + '.')

This operation filters the data from 10283 to 8843.


#### loading user surveys

In [53]:
user_surveys = pd.read_pickle('user_surveys.pkl')
user_surveys.head()

,urbanSuburbRural,barRestaurant,car,school,church,office,factory,home,kitchen,nature,cityStreet,shoppingGroceries,watchingMedia,listeningMusic,received,email,USER_ID
0,1,1,3,4,0,1,0,4,3,0,1,2,4,3,2018-02-14T21:56:52.755Z,NaN,alVWYiah1NGxN9FA9mRqRJWxy7ZZKRD2
1,1,2,3,4,4,0,0,4,2,0,3,2,4,1,2018-02-15T00:05:12.577Z,NaN,OtvPENAdy5wrsSZBGyKrblWraTHKds03
2,0,1,2,0,1,0,0,4,3,0,2,0,4,3,2018-02-23T21:01:10.288Z,mturker,YouPq79c0KEEMi3evNn9bYGyovJfLLOM
3,0,0,3,0,0,0,0,4,3,0,0,2,3,2,2018-02-23T21:01:35.768Z,mturker,WNzK3iaaokwedD7V6fKIjFIiZjBvgw0x
4,0,2,3,4,2,3,0,4,3,4,3,2,4,4,2018-02-23T21:02:33.810Z,mturker,c5k06ajbbB3qFfmJTgQAzRdUxAmXSoDd
